In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [1]:
class QuantumReservoirComputing:
    def __init__(self, n_qubits=4, depth=3, alpha=1.0):
        self.n_qubits = n_qubits
        self.depth = depth
        self.alpha = alpha
        self.simulator = AerSimulator()
        self.readout_model = Ridge(alpha=alpha)
        self.scaler = StandardScaler()

        # リザバー回路を事前に構築(固定)
        self.reservoir_circuit = self._build_reservoir()

    def _build_reservoir(self):
        """固定されたリザバー回路を構築"""
        qc = QuantumCircuit(self.n_qubits)
        np.random.seed(42)  # 再現性

        for d in range(self.depth):
            # ランダムな単一量子ビット回転
            for i in range(self.n_qubits):
                qc.rx(np.random.uniform(0, 2*np.pi), i)
                qc.ry(np.random.uniform(0, 2*np.pi), i)
                qc.rz(np.random.uniform(0, 2*np.pi), i)

            # エンタングルメント
            for i in range(self.n_qubits-1):
                qc.cx(i, i+1)
            qc.cx(self.n_qubits-1, 0)

        return qc

    def _encode_input(self, qc, x):
        """入力データを量子状態にエンコード"""
        for i in range(min(len(x), self.n_qubits)):
            qc.ry(x[i] * np.pi, i)
        return qc

    def _extract_features(self, qc, shots=1000):
        """量子回路を測定して特徴を抽出"""
        qc_copy = qc.copy()
        qc_copy.measure_all()

        job = self.simulator.run(qc_copy, shots=shots)
        counts = job.result().get_counts()

        # 確率分布を特徴ベクトルに変換
        n_states = 2**self.n_qubits
        features = np.zeros(n_states)

        for bitstring, count in counts.items():
            idx = int(bitstring, 2)
            features[idx] = count / shots

        return features

    def transform(self, X):
        """
        データセットを量子特徴に変換
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        
        Returns:
        --------
        quantum_features : array, shape (n_samples, 2^n_qubits)
        """
        quantum_features = []
        
        print(f"Processing {len(X)} samples...")
        for i, x in enumerate(X):
            if (i+1) % 10 == 0:
                print(f"  {i+1}/{len(X)} samples processed")
            
            # 新しい回路を作成
            qc = QuantumCircuit(self.n_qubits)
            
            # 入力をエンコード
            qc = self._encode_input(qc, x)
            
            # リザバー回路を適用
            qc = qc.compose(self.reservoir_circuit)
            
            # 特徴を抽出
            features = self._extract_features(qc)
            quantum_features.append(features)
        
        return np.array(quantum_features)
    
    def fit(self, X, y):
        """
        量子特徴を抽出して出力層を学習
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples,) or (n_samples, n_outputs)
        """
        # データを正規化
        X_scaled = self.scaler.fit_transform(X)
        
        # 量子特徴に変換
        quantum_features = self.transform(X_scaled)
        
        # 出力層を学習
        self.readout_model.fit(quantum_features, y)
        
        return self
    
    def predict(self, X):
        """
        予測を実行
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        
        Returns:
        --------
        predictions : array, shape (n_samples,) or (n_samples, n_outputs)
        """
        # データを正規化
        X_scaled = self.scaler.transform(X)
        
        # 量子特徴に変換
        quantum_features = self.transform(X_scaled)
        
        # 予測
        predictions = self.readout_model.predict(quantum_features)
        
        return predictions

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns

def plot_comprehensive_comparison(y_test, y_pred_dict, feature_names=None, save_path='prediction_comparison.png'):
    """
    予測値と理論値を包括的に比較するプロット
    
    Parameters:
    -----------
    y_test : array, shape (n_samples,) or (n_samples, n_features)
        真の値
    y_pred_dict : dict
        {'model_name': y_pred} の形式
        例: {'Linear Regression': y_pred_lr, 'QRC': y_pred_qrc}
    feature_names : list, optional
        各特徴量(列)の名前
    save_path : str
        保存先のパス
    """
    
    # データの形状を確認
    if y_test.ndim == 1:
        y_test = y_test.reshape(-1, 1)
    
    n_samples, n_features = y_test.shape
    n_models = len(y_pred_dict)
    
    # 特徴量名の設定
    if feature_names is None:
        feature_names = [f'Feature {i}' for i in range(n_features)]
    
    # 各モデルの予測値も2次元に
    for model_name in y_pred_dict:
        if y_pred_dict[model_name].ndim == 1:
            y_pred_dict[model_name] = y_pred_dict[model_name].reshape(-1, 1)
    
    # ===== プロット1: 時系列プロット =====
    n_cols = min(3, n_features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig1, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
    if n_features == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    colors = plt.cm.tab10(np.linspace(0, 1, n_models + 1))
    
    for feat_idx in range(n_features):
        ax = axes[feat_idx]
        
        # 真の値
        ax.plot(y_test[:, feat_idx], 
                label='True Value', 
                color=colors[0], 
                linewidth=2, 
                marker='o', 
                markersize=4,
                alpha=0.7)
        
        # 各モデルの予測
        for model_idx, (model_name, y_pred) in enumerate(y_pred_dict.items()):
            ax.plot(y_pred[:, feat_idx], 
                    label=model_name, 
                    color=colors[model_idx + 1],
                    linewidth=1.5,
                    marker='x',
                    markersize=4,
                    alpha=0.7)
        
        ax.set_xlabel('Time Step', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.set_title(f'{feature_names[feat_idx]}', fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=8)
        ax.grid(True, alpha=0.3)
    
    # 空のサブプロットを非表示
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path.replace('.png', '_timeseries.png'), dpi=300, bbox_inches='tight')
    print(f"Time series plot saved to '{save_path.replace('.png', '_timeseries.png')}'")
    
    # ===== プロット2: 散布図(True vs Predicted) =====
    fig2, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
    if n_features == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for feat_idx in range(n_features):
        ax = axes[feat_idx]
        
        # 各モデルの散布図
        for model_idx, (model_name, y_pred) in enumerate(y_pred_dict.items()):
            ax.scatter(y_test[:, feat_idx], 
                      y_pred[:, feat_idx], 
                      label=model_name,
                      color=colors[model_idx + 1],
                      alpha=0.6,
                      s=50,
                      edgecolors='black',
                      linewidth=0.5)
        
        # 理想的な予測線(y=x)
        min_val = min(y_test[:, feat_idx].min(), 
                     min([y_pred[:, feat_idx].min() for y_pred in y_pred_dict.values()]))
        max_val = max(y_test[:, feat_idx].max(),
                     max([y_pred[:, feat_idx].max() for y_pred in y_pred_dict.values()]))
        
        ax.plot([min_val, max_val], [min_val, max_val], 
                'r--', linewidth=2, label='Ideal', alpha=0.8)
        
        ax.set_xlabel('True Value', fontsize=10)
        ax.set_ylabel('Predicted Value', fontsize=10)
        ax.set_title(f'{feature_names[feat_idx]}', fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=8)
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal', adjustable='box')
    
    # 空のサブプロットを非表示
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path.replace('.png', '_scatter.png'), dpi=300, bbox_inches='tight')
    print(f"Scatter plot saved to '{save_path.replace('.png', '_scatter.png')}'")
    
    # ===== プロット3: 誤差分布 =====
    fig3, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
    if n_features == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for feat_idx in range(n_features):
        ax = axes[feat_idx]
        
        # 各モデルの誤差のヒストグラム
        for model_idx, (model_name, y_pred) in enumerate(y_pred_dict.items()):
            errors = y_test[:, feat_idx] - y_pred[:, feat_idx]
            ax.hist(errors, 
                   bins=20, 
                   alpha=0.6, 
                   label=f'{model_name}\n(μ={errors.mean():.4f}, σ={errors.std():.4f})',
                   color=colors[model_idx + 1],
                   edgecolor='black')
        
        ax.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Zero Error')
        ax.set_xlabel('Error (True - Predicted)', fontsize=10)
        ax.set_ylabel('Frequency', fontsize=10)
        ax.set_title(f'{feature_names[feat_idx]}', fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=7)
        ax.grid(True, alpha=0.3, axis='y')
    
    # 空のサブプロットを非表示
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path.replace('.png', '_error_dist.png'), dpi=300, bbox_inches='tight')
    print(f"Error distribution plot saved to '{save_path.replace('.png', '_error_dist.png')}'")
    
    # ===== プロット4: メトリクス比較 =====
    fig4, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    metrics_data = {
        'MSE': [],
        'MAE': [],
        'R²': []
    }
    
    model_names_list = list(y_pred_dict.keys())
    
    # 各特徴量の平均メトリクスを計算
    for model_name, y_pred in y_pred_dict.items():
        mse_list = []
        mae_list = []
        r2_list = []
        
        for feat_idx in range(n_features):
            mse = mean_squared_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            mae = mean_absolute_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            r2 = r2_score(y_test[:, feat_idx], y_pred[:, feat_idx])
            
            mse_list.append(mse)
            mae_list.append(mae)
            r2_list.append(r2)
        
        metrics_data['MSE'].append(np.mean(mse_list))
        metrics_data['MAE'].append(np.mean(mae_list))
        metrics_data['R²'].append(np.mean(r2_list))
    
    # MSE
    axes[0].bar(model_names_list, metrics_data['MSE'], color=colors[1:n_models+1], alpha=0.7, edgecolor='black')
    axes[0].set_ylabel('MSE', fontsize=12)
    axes[0].set_title('Mean Squared Error', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    axes[0].tick_params(axis='x', rotation=45)
    
    # MAE
    axes[1].bar(model_names_list, metrics_data['MAE'], color=colors[1:n_models+1], alpha=0.7, edgecolor='black')
    axes[1].set_ylabel('MAE', fontsize=12)
    axes[1].set_title('Mean Absolute Error', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    axes[1].tick_params(axis='x', rotation=45)
    
    # R²
    axes[2].bar(model_names_list, metrics_data['R²'], color=colors[1:n_models+1], alpha=0.7, edgecolor='black')
    axes[2].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[2].set_ylabel('R²', fontsize=12)
    axes[2].set_title('R² Score', fontsize=14, fontweight='bold')
    axes[2].grid(True, alpha=0.3, axis='y')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(save_path.replace('.png', '_metrics.png'), dpi=300, bbox_inches='tight')
    print(f"Metrics comparison plot saved to '{save_path.replace('.png', '_metrics.png')}'")
    
    # ===== プロット5: 詳細メトリクステーブル =====
    fig5, ax = plt.subplots(figsize=(12, max(6, n_features * 0.8)))
    ax.axis('tight')
    ax.axis('off')
    
    # テーブルデータの作成
    table_data = []
    headers = ['Feature', 'Model', 'MSE', 'MAE', 'R²', 'RMSE']
    
    for feat_idx in range(n_features):
        for model_name, y_pred in y_pred_dict.items():
            mse = mean_squared_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            mae = mean_absolute_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            r2 = r2_score(y_test[:, feat_idx], y_pred[:, feat_idx])
            rmse = np.sqrt(mse)
            
            table_data.append([
                feature_names[feat_idx],
                model_name,
                f'{mse:.6f}',
                f'{mae:.6f}',
                f'{r2:.4f}',
                f'{rmse:.6f}'
            ])
    
    table = ax.table(cellText=table_data, 
                     colLabels=headers,
                     cellLoc='center',
                     loc='center',
                     colWidths=[0.15, 0.2, 0.15, 0.15, 0.15, 0.15])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    # ヘッダーのスタイル
    for i in range(len(headers)):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # 行の色分け
    for i in range(1, len(table_data) + 1):
        if i % 2 == 0:
            for j in range(len(headers)):
                table[(i, j)].set_facecolor('#f0f0f0')
    
    plt.title('Detailed Metrics Table', fontsize=16, fontweight='bold', pad=20)
    plt.savefig(save_path.replace('.png', '_table.png'), dpi=300, bbox_inches='tight')
    print(f"Metrics table saved to '{save_path.replace('.png', '_table.png')}'")
    
    plt.close('all')
    
    # ===== コンソール出力 =====
    print("\n" + "="*70)
    print("PREDICTION COMPARISON SUMMARY")
    print("="*70)
    
    for feat_idx in range(n_features):
        print(f"\n{feature_names[feat_idx]}:")
        print("-" * 70)
        print(f"{'Model':<20} {'MSE':>12} {'MAE':>12} {'R²':>12} {'RMSE':>12}")
        print("-" * 70)
        
        for model_name, y_pred in y_pred_dict.items():
            mse = mean_squared_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            mae = mean_absolute_error(y_test[:, feat_idx], y_pred[:, feat_idx])
            r2 = r2_score(y_test[:, feat_idx], y_pred[:, feat_idx])
            rmse = np.sqrt(mse)
            
            print(f"{model_name:<20} {mse:>12.6f} {mae:>12.6f} {r2:>12.4f} {rmse:>12.6f}")
    
    print("\n" + "="*70)
    print("AVERAGE ACROSS ALL FEATURES:")
    print("-" * 70)
    print(f"{'Model':<20} {'MSE':>12} {'MAE':>12} {'R²':>12} {'RMSE':>12}")
    print("-" * 70)
    
    for model_name, y_pred in y_pred_dict.items():
        mse_avg = np.mean([mean_squared_error(y_test[:, i], y_pred[:, i]) for i in range(n_features)])
        mae_avg = np.mean([mean_absolute_error(y_test[:, i], y_pred[:, i]) for i in range(n_features)])
        r2_avg = np.mean([r2_score(y_test[:, i], y_pred[:, i]) for i in range(n_features)])
        rmse_avg = np.sqrt(mse_avg)
        
        print(f"{model_name:<20} {mse_avg:>12.6f} {mae_avg:>12.6f} {r2_avg:>12.4f} {rmse_avg:>12.6f}")
    
    print("="*70)


def plot_single_feature_detailed(y_test, y_pred_dict, feature_idx=0, feature_name=None, save_path='detailed_comparison.png'):
    """
    単一の特徴量に対する詳細な比較プロット
    
    Parameters:
    -----------
    y_test : array, shape (n_samples,) or (n_samples, n_features)
    y_pred_dict : dict
    feature_idx : int
        表示する特徴量のインデックス
    feature_name : str, optional
    save_path : str
    """
    
    if y_test.ndim == 1:
        y_true = y_test
    else:
        y_true = y_test[:, feature_idx]
    
    if feature_name is None:
        feature_name = f'Feature {feature_idx}'
    
    # 各モデルの予測値を抽出
    predictions = {}
    for model_name, y_pred in y_pred_dict.items():
        if y_pred.ndim == 1:
            predictions[model_name] = y_pred
        else:
            predictions[model_name] = y_pred[:, feature_idx]
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(predictions) + 1))
    
    # 1. 時系列プロット(大きめ)
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(y_true, label='True Value', color=colors[0], linewidth=2.5, marker='o', markersize=5, alpha=0.8)
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        ax1.plot(y_pred, label=model_name, color=colors[idx+1], linewidth=2, marker='x', markersize=5, alpha=0.7)
    ax1.set_xlabel('Time Step', fontsize=12)
    ax1.set_ylabel('Value', fontsize=12)
    ax1.set_title(f'Time Series: {feature_name}', fontsize=14, fontweight='bold')
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # 2. 散布図
    ax2 = fig.add_subplot(gs[1, 0])
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        ax2.scatter(y_true, y_pred, label=model_name, color=colors[idx+1], alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    
    min_val = min(y_true.min(), min([p.min() for p in predictions.values()]))
    max_val = max(y_true.max(), max([p.max() for p in predictions.values()]))
    ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Ideal')
    ax2.set_xlabel('True Value', fontsize=10)
    ax2.set_ylabel('Predicted Value', fontsize=10)
    ax2.set_title('Scatter Plot', fontsize=12, fontweight='bold')
    ax2.legend(loc='best', fontsize=8)
    ax2.grid(True, alpha=0.3)
    ax2.set_aspect('equal', adjustable='box')
    
    # 3. 誤差の時系列
    ax3 = fig.add_subplot(gs[1, 1])
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        errors = y_true - y_pred
        ax3.plot(errors, label=model_name, color=colors[idx+1], linewidth=1.5, alpha=0.7)
    ax3.axhline(y=0, color='r', linestyle='--', linewidth=2)
    ax3.set_xlabel('Time Step', fontsize=10)
    ax3.set_ylabel('Error (True - Predicted)', fontsize=10)
    ax3.set_title('Error Over Time', fontsize=12, fontweight='bold')
    ax3.legend(loc='best', fontsize=8)
    ax3.grid(True, alpha=0.3)
    
    # 4. 誤差分布
    ax4 = fig.add_subplot(gs[1, 2])
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        errors = y_true - y_pred
        ax4.hist(errors, bins=20, alpha=0.6, label=f'{model_name}\nμ={errors.mean():.4f}\nσ={errors.std():.4f}',
                color=colors[idx+1], edgecolor='black')
    ax4.axvline(x=0, color='r', linestyle='--', linewidth=2)
    ax4.set_xlabel('Error', fontsize=10)
    ax4.set_ylabel('Frequency', fontsize=10)
    ax4.set_title('Error Distribution', fontsize=12, fontweight='bold')
    ax4.legend(loc='best', fontsize=7)
    ax4.grid(True, alpha=0.3, axis='y')
    
    # 5. 残差プロット
    ax5 = fig.add_subplot(gs[2, 0])
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        residuals = y_true - y_pred
        ax5.scatter(y_pred, residuals, label=model_name, color=colors[idx+1], alpha=0.6, s=50)
    ax5.axhline(y=0, color='r', linestyle='--', linewidth=2)
    ax5.set_xlabel('Predicted Value', fontsize=10)
    ax5.set_ylabel('Residual', fontsize=10)
    ax5.set_title('Residual Plot', fontsize=12, fontweight='bold')
    ax5.legend(loc='best', fontsize=8)
    ax5.grid(True, alpha=0.3)
    
    # 6. QQプロット
    from scipy import stats
    ax6 = fig.add_subplot(gs[2, 1])
    for idx, (model_name, y_pred) in enumerate(predictions.items()):
        errors = y_true - y_pred
        stats.probplot(errors, dist="norm", plot=ax6)
        ax6.get_lines()[idx*2].set_color(colors[idx+1])
        ax6.get_lines()[idx*2].set_label(model_name)
    ax6.set_title('Q-Q Plot', fontsize=12, fontweight='bold')
    ax6.legend(loc='best', fontsize=8)
    ax6.grid(True, alpha=0.3)
    
    # 7. メトリクスバー
    ax7 = fig.add_subplot(gs[2, 2])
    metrics = {'MSE': [], 'MAE': [], 'R²': []}
    model_names = list(predictions.keys())
    
    for model_name, y_pred in predictions.items():
        metrics['MSE'].append(mean_squared_error(y_true, y_pred))
        metrics['MAE'].append(mean_absolute_error(y_true, y_pred))
        metrics['R²'].append(r2_score(y_true, y_pred))
    
    x = np.arange(len(model_names))
    width = 0.25
    
    ax7_twin1 = ax7.twinx()
    ax7_twin2 = ax7.twinx()
    ax7_twin2.spines['right'].set_position(('outward', 60))
    
    ax7.bar(x - width, metrics['MSE'], width, label='MSE', color='skyblue', alpha=0.8)
    ax7_twin1.bar(x, metrics['MAE'], width, label='MAE', color='lightcoral', alpha=0.8)
    ax7_twin2.bar(x + width, metrics['R²'], width, label='R²', color='lightgreen', alpha=0.8)
    
    ax7.set_xlabel('Model', fontsize=10)
    ax7.set_ylabel('MSE', fontsize=10, color='skyblue')
    ax7_twin1.set_ylabel('MAE', fontsize=10, color='lightcoral')
    ax7_twin2.set_ylabel('R²', fontsize=10, color='lightgreen')
    ax7.set_title('Metrics Comparison', fontsize=12, fontweight='bold')
    ax7.set_xticks(x)
    ax7.set_xticklabels(model_names, rotation=45, ha='right')
    
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Detailed comparison plot saved to '{save_path}'")
    plt.close()


# ===== 使用例 =====
if __name__ == "__main__":
    # サンプルデータ
    np.random.seed(42)
    n_samples = 100
    n_features = 3
    
    # 真の値
    y_test = np.random.randn(n_samples, n_features) * 10 + 50
    
    # モデルの予測値(シミュレーション)
    y_pred_lr = y_test + np.random.randn(n_samples, n_features) * 2
    y_pred_qrc = y_test + np.random.randn(n_samples, n_features) * 3
    y_pred_rf = y_test + np.random.randn(n_samples, n_features) * 1.5
    
    # 予測値の辞書
    predictions = {
        'Linear Regression': y_pred_lr,
        'QRC': y_pred_qrc,
        'Random Forest': y_pred_rf
    }
    
    # 特徴量名
    feature_names = ['Tenor_10_Mat_30', 'Tenor_15_Mat_30', 'Tenor_20_Mat_30']
    
    # 包括的な比較プロット
    plot_comprehensive_comparison(y_test, predictions, feature_names, save_path='comparison.png')
    
    # 詳細プロット(最初の特徴量)
    plot_single_feature_detailed(y_test, predictions, feature_idx=0, 
                                 feature_name='Tenor_10_Mat_30', 
                                 save_path='detailed_comparison.png')

Time series plot saved to 'comparison_timeseries.png'
Scatter plot saved to 'comparison_scatter.png'
Error distribution plot saved to 'comparison_error_dist.png'
Metrics comparison plot saved to 'comparison_metrics.png'
Metrics table saved to 'comparison_table.png'

PREDICTION COMPARISON SUMMARY

Tenor_10_Mat_30:
----------------------------------------------------------------------
Model                         MSE          MAE           R²         RMSE
----------------------------------------------------------------------
Linear Regression        3.656363     1.538679       0.9457     1.912162
QRC                      7.298574     2.129872       0.8916     2.701587
Random Forest            2.388992     1.241258       0.9645     1.545637

Tenor_15_Mat_30:
----------------------------------------------------------------------
Model                         MSE          MAE           R²         RMSE
----------------------------------------------------------------------
Linear Regression 